<a href="https://colab.research.google.com/github/aleeepassarelli/scientific-validation-hub/blob/main/notebooks/04_archiving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏛️ Lab 04: Digital Archiving com Zenodo (Sandbox)

**Framework:** Scientific Validation Hub
**Ferramenta:** [Zenodo](https://zenodo.org) (CERN)
**Objetivo:** Gerar um DOI (Digital Object Identifier) para seu código ou dataset, tornando-o imutável e citável academicamente.

---

### ⚡ Quick Start (Modo Assistido)
Este laboratório usa o **Zenodo Sandbox** (ambiente de testes), então não vamos criar registros reais na ciência, apenas simular.

1.  **Obtenha um Token (Opcional mas Recomendado):**
    * Vá para [sandbox.zenodo.org](https://sandbox.zenodo.org/account/settings/applications/).
    * Logue com seu GitHub.
    * Clique em "New Token", dê permissão `deposit:write` e `deposit:actions`, e copie a chave.
    * Cole a chave na variável `ZENODO_TOKEN` abaixo.
    * *(Se não quiser fazer isso, deixe vazio; o script rodará em Modo Simulação).*

2.  **Execute:** Rode as células para "depositar" um arquivo científico.
3.  **Audite:** Abra o Assistente Gemini e cole:

> "Atue como Bibliotecário Acadêmico.
> Analise o JSON de resposta do Zenodo abaixo.
> 1. O depósito foi criado com sucesso (status 201)?
> 2. Existe um campo 'metadata.prereserve_doi.doi'? Qual é o DOI reservado?
> 3. Explique a importância de ter um DOI começando com '10.5281' para a reprodutibilidade."

In [ ]:
import requests
import json
import os

# --- ÁREA DE CONFIGURAÇÃO ---
# Cole seu token do Sandbox Zenodo aqui entre aspas.
# Se deixar vazio "", rodaremos uma simulação offline.
ZENODO_TOKEN = ""

# Metadados do seu "Paper" ou Dataset
METADATA = {
    "metadata": {
        "title": "Dataset de Validação de Agentes SLE v1.1 (Teste)",
        "upload_type": "dataset",
        "description": "Dados gerados automaticamente para teste de arquivamento digital.",
        "creators": [
            {"name": "Hacker Semântico", "affiliation": "Scientific Validation Hub"}
        ]
    }
}

# Criando um arquivo dummy para upload
with open("meu_artigo_final.csv", "w") as f:
    f.write("id,score\n1,0.99\n2,0.98")

print("✅ Ambiente configurado. Arquivo 'meu_artigo_final.csv' pronto para upload.")

In [ ]:
def deposit_to_zenodo(token, metadata, filename):
    # Endpoint do Sandbox (Não afeta a ciência real)
    base_url = "https://sandbox.zenodo.org/api/deposit/depositions"
    params = {'access_token': token}
    headers = {"Content-Type": "application/json"}

    print(f"📡 Conectando ao CERN (Zenodo Sandbox)...")

    # 1. Criar o Depósito (O "Envelope" vazio)
    r = requests.post(base_url, params=params, json=metadata, headers=headers)

    if r.status_code != 201:
        return {"error": r.status_code, "message": r.json()}

    deposition_id = r.json()['id']
    print(f"   --> Depósito criado! ID: {deposition_id}")

    # 2. Upload do Arquivo (Colocando o papel no envelope)
    bucket_url = r.json()['links']['bucket']
    print(f"   --> Fazendo upload de {filename}...")

    with open(filename, "rb") as fp:
        r_file = requests.put(
            f"{bucket_url}/{filename}",
            data=fp,
            params=params,
        )

    return r.json()

def simulate_deposit():
    print("⚠️ MODO SIMULAÇÃO (Sem Token Real)")
    print("   --> Simulando handshake com Zenodo...")
    print("   --> Gerando DOI fake para demonstração...")
    return {
        "id": 123456,
        "metadata": {
            "prereserve_doi": {"doi": "10.5281/zenodo.simulate123"},
            "title": METADATA["metadata"]["title"]
        },
        "simulation": True
    }

# Lógica de Execução
if len(ZENODO_TOKEN) > 10:
    response = deposit_to_zenodo(ZENODO_TOKEN, METADATA, "meu_artigo_final.csv")
else:
    response = simulate_deposit()

In [ ]:
# Exibindo o resultado para o Gemini Auditar
print("\n" + "="*60)
print("🏛️ RELATÓRIO DE ARQUIVAMENTO DIGITAL")
print("="*60)

if "error" in response:
    print(f"❌ FALHA NO DEPÓSITO: {response['message']}")
else:
    # Extraindo o DOI (A Placa do Carro)
    doi = response['metadata']['prereserve_doi']['doi']

    print(f"✅ SUCESSO! Objeto Científico Criado.")
    print(f"   Título: {response['metadata']['title']}")
    print(f"   DOI RESERVADO: {doi}")

    if "simulation" in response:
        print("\n   [NOTA: Este é um DOI simulado. Adicione um Token real para gerar um válido]")
    else:
        print(f"\n   Link de Rascunho: https://sandbox.zenodo.org/deposit/{response['id']}")
        print("   (Para publicar oficialmente, você precisaria clicar em 'Publish' no site)")

print("="*60)
print("\n🤖 PRÓXIMO PASSO: Peça para o Gemini validar se o formato do DOI está correto (começa com 10.5281).")